In [7]:
# --- 0. 安装必要的库 ---
# 在Kaggle环境中，您需要先安装sentence-transformers
# 确保您的Kaggle Notebook已在 "Settings" -> "Internet" 中开启网络
!pip install -q sentence-transformers transformers scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 14.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 3.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 6.3 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependen

In [9]:
# --- 1. 导入库 ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib.colors as mcolors
import os
import sys
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE  # 用于降维可视化
from sklearn.feature_extraction.text import TfidfVectorizer  # 用于提取关键词
try:
    from sentence_transformers import SentenceTransformer
except ImportError:
    print("错误：无法导入 SentenceTransformer。请先在 Kaggle 中执行：!pip install -q sentence-transformers transformers")
    sys.exit()

# --- 2. 设置字体和文件路径 ---
font_path_chinese = "/kaggle/input/nssfc-2025/simkai.ttf"
font_path_english = "/kaggle/input/nssfc-2025/times.ttf"
data_file = "/kaggle/input/nssfc-2025/nssfc-fenci.csv"

# 字体注册
if os.path.exists(font_path_chinese):
    ch_font = fm.FontProperties(fname=font_path_chinese)
    fm.fontManager.addfont(font_path_chinese)
else:
    print(f"警告: 中文字体 '{font_path_chinese}' 未找到。")
    ch_font = fm.FontProperties()

if os.path.exists(font_path_english):
    en_font = fm.FontProperties(fname=font_path_english)
    fm.fontManager.addfont(font_path_english)
else:
    print(f"警告: 英文字体 '{font_path_english}' 未找到。")
    en_font = fm.FontProperties()

plt.rcParams['axes.unicode_minus'] = False

# --- 3. 加载与筛选数据 ---
print(f"正在从 '{data_file}' 加载数据...")
try:
    df = pd.read_csv(data_file, dtype=str, encoding="utf-8")
except FileNotFoundError:
    print(f"错误: 数据文件 '{data_file}' 未找到。")
    sys.exit()
except Exception as e:
    print(f"读取 CSV 文件时出错: {e}")
    sys.exit()

df.columns = [str(c).strip() for c in df.columns]

# 检查必要列
required_cols = {"课题名称", "涉及学科"}
if not required_cols.issubset(df.columns):
    print(f"错误：缺少列 {required_cols}，当前列为：{list(df.columns)}")
    sys.exit()

# 只保留管理学项目
df = df[df["涉及学科"] == "管理学"].copy()
if df.empty:
    print("错误：未找到涉及学科为 '管理学' 的记录。")
    sys.exit()

# 提取文本列
titles = df["课题名称"].dropna().astype(str).tolist()
if not titles:
    print("错误：'课题名称' 列为空。")
    sys.exit()

print(f"已加载 {len(titles)} 条 '管理学' 项目。示例前3条：")
for t in texts[:3]:
    print(" -", t)


# --- 4. 加载嵌入模型并生成编码 ---
model_name = "jinaai/jina-embeddings-v2-base-zh"
print(f"正在从 Hugging Face 加载模型：{model_name} ...")
try:
    model = SentenceTransformer(model_name, trust_remote_code=True)
    print("模型加载成功。")
except Exception as e:
    print(f"加载模型时出错: {e}")
    print("请确认网络已开启。")
    sys.exit()

print(f"正在为 {len(titles)} 个标题生成编码（可能需要一些时间）...")
embeddings = model.encode(titles, show_progress_bar=True)
print(f"编码完成。shape = {embeddings.shape}")

# --- 5. K-means 聚类与评估 ---
K_range = range(2, 16)
silhouette_scores = []
ch_scores = []
db_scores_inv = []

print("开始 K-means 聚类与评估循环 (K=2 到 15)...")
for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(embeddings)

    sil = silhouette_score(embeddings, labels)
    silhouette_scores.append(sil)

    ch = calinski_harabasz_score(embeddings, labels)
    ch_scores.append(ch)

    db = davies_bouldin_score(embeddings, labels)
    db_scores_inv.append(1 - db)

    print(f"  K={k}: Silhouette={sil:.3f}, CH={ch:.0f}, 1-DB={1-db:.3f}")

print("聚类评估完成。")


正在从 '/kaggle/input/nssfc-2025/nssfc-fenci.csv' 加载数据...
已加载 590 条 '管理学' 项目。示例前3条：
 - 我国 战略 腹地 安全 韧性 演化 机理 治理 策略 研究
 - 数智 技术 驱动 公共 管理 范式 转型 理论 重构 实践 路径 计算 实验 研究
 - 人工 智能 赋能 政府 监管 效能 挑战 策略 研究
正在从 Hugging Face 加载模型：jinaai/jina-embeddings-v2-base-zh ...
模型加载成功。
正在为 590 个标题生成编码（可能需要一些时间）...


Batches:   0%|          | 0/19 [00:00<?, ?it/s]

编码完成。shape = (590, 768)
开始 K-means 聚类与评估循环 (K=2 到 15)...
  K=2: Silhouette=0.048, CH=28, 1-DB=-3.590
  K=3: Silhouette=0.031, CH=22, 1-DB=-3.498
  K=4: Silhouette=0.036, CH=20, 1-DB=-3.252
  K=5: Silhouette=0.037, CH=18, 1-DB=-3.093
  K=6: Silhouette=0.037, CH=17, 1-DB=-2.943
  K=7: Silhouette=0.037, CH=16, 1-DB=-2.793
  K=8: Silhouette=0.042, CH=15, 1-DB=-2.673
  K=9: Silhouette=0.045, CH=14, 1-DB=-2.527
  K=10: Silhouette=0.039, CH=13, 1-DB=-2.620
  K=11: Silhouette=0.043, CH=13, 1-DB=-2.487
  K=12: Silhouette=0.040, CH=12, 1-DB=-2.454
  K=13: Silhouette=0.043, CH=12, 1-DB=-2.365
  K=14: Silhouette=0.038, CH=11, 1-DB=-2.296
  K=15: Silhouette=0.044, CH=11, 1-DB=-2.246
聚类评估完成。


In [10]:
from matplotlib.ticker import StrMethodFormatter

# --- 6. 绘制评估指标图表（微调版） ---
print("正在生成评估图表...")
fig, ax1 = plt.subplots(figsize=(15, 8))

# 标题（通用，不依赖 target_discipline）
plt.title('项目聚类性能评估 (K=2 to 15)', fontproperties=ch_font, fontsize=20, pad=20)

# X轴
K_list = list(K_range)  # 确保是 list，便于索引与标注
ax1.set_xlabel('簇数量 (K)', fontproperties=ch_font, fontsize=14)
ax1.set_xticks(K_list)  # 确保X轴刻度为整数
plt.setp(ax1.get_xticklabels(), fontproperties=en_font, fontsize=12)

# Y轴1：Silhouette & 1-DB
color1 = 'tab:blue'
ax1.set_ylabel('Silhouette / (1 - DB) 得分', fontproperties=ch_font, fontsize=14, color=color1)

# 注意：ASCII '-'，避免 U+2212
line1 = ax1.plot(K_list, silhouette_scores, marker='o', linestyle='-', color='tab:blue',
                 linewidth=2, markersize=7, label='Silhouette 系数')
line2 = ax1.plot(K_list, db_scores_inv,  marker='s', linestyle='--', color='tab:cyan',
                 linewidth=2, markersize=6, label='1 - Davies-Bouldin 指数')

ax1.tick_params(axis='y', labelcolor=color1)
plt.setp(ax1.get_yticklabels(), fontproperties=en_font, fontsize=12)
ax1.grid(True, linestyle='--', alpha=0.7, axis='x')

# Y轴2：CH 指数（右轴）
ax2 = ax1.twinx()
color2 = 'tab:red'
ax2.set_ylabel('Calinski–Harabasz 指数', fontproperties=ch_font, fontsize=14, color=color2)
line3 = ax2.plot(K_list, ch_scores, marker='^', linestyle='-.', color=color2,
                 linewidth=2, markersize=7, label='Calinski–Harabasz 指数')
ax2.tick_params(axis='y', labelcolor=color2)
plt.setp(ax2.get_yticklabels(), fontproperties=en_font, fontsize=12)
ax2.yaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))  # 千分位

# --- 标注三项指标的“最佳 K” ---
best_k_sil = K_list[int(np.argmax(silhouette_scores))]
best_k_db  = K_list[int(np.argmax(db_scores_inv))]
best_k_ch  = K_list[int(np.argmax(ch_scores))]

# 垂直虚线（用不同样式区分）
for k, ls, c in [(best_k_sil, ':', 'tab:blue'),
                 (best_k_db,  ':', 'tab:cyan'),
                 (best_k_ch,  ':', 'tab:red')]:
    ax1.axvline(k, linestyle=ls, color=c, alpha=0.25)



# --- 图例放在图下方 ---
lines = line1 + line2 + line3
labels = [l.get_label() for l in lines]

fig.subplots_adjust(bottom=0.22)  # 为底部图例预留空间
legend = fig.legend(
    lines, labels,
    loc='lower center',
    ncol=3,
    frameon=False,
    prop=ch_font,
    bbox_to_anchor=(0.5, 0.02)
)

fig.tight_layout(rect=[0, 0.05, 1, 1])  # 与底部图例协调（保留 5% 底边）

# --- 7. 显示和保存图表 ---
output_filename = 'management_clustering_metrics.png'
try:
    fig.savefig(output_filename, dpi=300, bbox_inches='tight')
    print(f"图表已成功保存为: {output_filename} (位于 /kaggle/working/ 目录)")
except Exception as e:
    print(f"保存图表 '{output_filename}' 时出错: {e}")

try:
    print("正在显示图表: 聚类评估")
    plt.show()
except Exception as e:
    print(f"显示图表时出错: {e}")

plt.close(fig)
print("所有任务完成。")


正在生成评估图表...
图表已成功保存为: management_clustering_metrics.png (位于 /kaggle/working/ 目录)
正在显示图表: 聚类评估


所有任务完成。
